In [16]:
#Extract dataset from original for testing 
##first dataset

import pandas as pd

# Read the dataset
df = pd.read_csv('C:\\Users\\prane\\Documents\\Univ_of_Bristol\\mini-project\\backup-file\\fake_transactional_data_24.csv')

# Convert not_happened_yet_date to datetime
df['not_happened_yet_date'] = pd.to_datetime(df['not_happened_yet_date'], format='%d/%m/%Y')

# Take 200 unique from_totally_fake_account values
unique_accounts = df['from_totally_fake_account'].unique()[:200]

# Filter data to include only transactions from these 100 accounts
filtered_df = df[df['from_totally_fake_account'].isin(unique_accounts)]

# Output the final DataFrame
print(filtered_df.shape)
#filtered_df.to_csv('C:\\Users\\prane\\Documents\\Univ_of_Bristol\\mini-project\\filtered-result.csv', index=False)



(261716, 4)


In [2]:
##new dataset -2L records so no need to extract subset

import pandas as pd

# Read the dataset
df = pd.read_csv('C:\\Users\\prane\\Documents\\Univ_of_Bristol\\mini-project\\simulated_transaction_2024.csv')
print(df.shape)

df.describe()


(230596, 7)


,Account No,Balance,Amount,Third Party Account No
count,2.303720e+05,230350.000000,230387.000000,6.832000e+03
mean,5.505993e+08,3609.351057,6.437717,5.370263e+08
std,2.560610e+08,5379.331174,508.200101,2.596927e+08
min,1.015313e+08,-5790.983500,-12240.000000,1.019085e+08
25%,3.314502e+08,373.453932,-74.000000,3.114679e+08
50%,5.501698e+08,1686.009850,-28.700000,5.381491e+08
75%,7.682718e+08,5215.031852,-12.060000,7.485097e+08
max,9.997527e+08,46804.158837,12240.000000,9.997527e+08


In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 230596 entries, 0 to 230595
Data columns (total 7 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   Date                    230367 non-null  object 
 1   Timestamp               230345 non-null  object 
 2   Account No              230372 non-null  float64
 3   Balance                 230350 non-null  float64
 4   Amount                  230387 non-null  float64
 5   Third Party Account No  6832 non-null    float64
 6   Third Party Name        223517 non-null  object 
dtypes: float64(4), object(3)
memory usage: 12.3+ MB


In [21]:
df.nunique()

Date                         340
Timestamp                   1401
Account No                   976
Balance                   229888
Amount                     17314
Third Party Account No       673
Third Party Name              83
dtype: int64

In [3]:
# Check for null values
print("Null values for features:")
print("Date:" + str(df['Date'].isnull().sum()))
print("Timestamp:" + str(df['Timestamp'].isnull().sum()))
print("Account No:" + str(df['Account No'].isnull().sum()))
print("Balance:" + str(df['Balance'].isnull().sum()))
print("Amount:" + str(df['Amount'].isnull().sum()))
print("Third Party Account No:" + str(df['Third Party Account No'].isnull().sum()))
print("Third Party Name:" + str(df['Third Party Name'].isnull().sum()))
print("")
print("----------------------------------------------")

# Remove records with null values for Date, Account No, Amount
df_cleaned = df.dropna(subset=['Date', 'Account No', 'Amount'])

# Remove records where both Third Party Account No and Third Party Name are null
df_cleaned = df_cleaned.dropna(subset=['Third Party Account No', 'Third Party Name'], how='all')
#print(df[(df['Third Party Account No'].isnull()) & (df['Third Party Name'].isnull())]) #247

# Print the shape of the cleaned DataFrame
print("Shape of cleaned DataFrame:", df_cleaned.shape)
#Original df shape: (230596, 7)
print("Total records dropped for null values:" + str(df.shape[0] - df_cleaned.shape[0]))

Null values for features:
Date:229
Timestamp:251
Account No:224
Balance:246
Amount:209
Third Party Account No:223764
Third Party Name:7079

----------------------------------------------
Shape of cleaned DataFrame: (229689, 7)
Total records dropped for null values:907


In [10]:
#testing


In [71]:
# Check for non-date values in feature Date

try:
    pd.to_datetime(df_cleaned['Date'], format='%d/%m/%Y', errors='raise')
    print("No unexpected values found in 'Date'.")
except ValueError as e:
    print(f"Unexpected values found in 'Date': {e}")

No unexpected values found in 'Date'.


In [43]:
#Validating lengths of from account no's
print(df_cleaned['Account No'].astype(str).str.split('.').str[0].apply(len).unique())
print(df_cleaned['Third Party Account No'].astype(str).str.split('.').str[0].apply(len).unique())

#print(df_cleaned[df_cleaned['Third Party Account No'].astype(str).str.split('.').str[0].apply(len) == 3])
#Here 'Third Party Account No' has account no. len 3 for NaN values which is expected.

[9]
[3 9]


In [46]:
# Validation for string values in Third Party Name: check if empty or special characters are present
import re

str_validation = df_cleaned['Third Party Name'][~df_cleaned['Third Party Name'].astype(str).str.match('^[a-zA-Z ]*$')]
if not str_validation.empty:
    print("Blank values or special characters found in 'Third Party Name':")
    print(str_validation.unique())
    print(len(str_validation))
else:
    print("No blank values or special characters found in 'Third Party Name'.")
    
#The values looks expected, so not dropping

Blank values or special characters found in 'Third Party Name':
['Rose & Crown' "Blackwell's" 'Coffee #1' 'Frankie & Bennies'
 'Mamas & Papas']
6274


In [48]:
#Distinct Third Party Name values for categorization

df_cleaned['Third Party Name'].unique() #count=83

array(['Westport Care Home', 'Barbiee Boutique', nan, 'Fat Face',
       'Lavender Primary', 'A Cut Above', 'Tesco', 'Brilliant Brushes',
       'Costa Coffee', 'Premier Finance', 'North Face', 'Craftastic',
       'Fitted Stitch', 'Selfridges', 'Starbucks', 'The Crown',
       'Cass Art', 'Green Park Academy', 'Rose & Crown', 'Kings Arms',
       'Sunny Care Nursery', 'A Yarn Story', 'Pets Corner',
       'Lavender Fields', 'Victoria Park', 'Halifax', 'LBG', 'Blizzard',
       'Xbox', 'Mojang Studios', 'PureGym', 'Disney', 'Netflix',
       'Grand Union BJJ', 'Amazon', 'SquareOnix', 'Deliveroo', 'JustEat',
       'Coop Local', 'Sainsbury', 'AMAZON', 'The Works', "Blackwell's",
       'Topshop', 'Matalan', 'Foyles', 'Wool', 'Hobby Lobby', 'Revella',
       'Sainsbury Local', 'Loosely Fitted', 'Stitch By Stitch',
       'Coffee #1', 'Hobbycraft', 'Lloyds Pharmacy', 'Gamestation', 'CeX',
       'Etsy', 'Five Senses Art', 'Sports Direct', 'Boots', 'Reebok',
       'JD Sports', 'Head', 'Fr

In [ ]:
Date                         340
Timestamp                   1401
Account No                   976
Balance                   229888
Amount                     17314
Third Party Account No       673
Third Party Name              83

In [67]:
#do not take as it should be relevant to business/classification as did earlier and not on overall transactions

#Clustering: Based on Customer lifetime value (CLV)
#CLV = avg revenue per transaction * avg transaction frequency

df_clv = df_cleaned.copy()
df_clv['Date'] = pd.to_datetime(df_clv['Date'], format='%d/%m/%Y')

# Calculate total revenue per customer
df_clv['total_revenue_per_customer'] = df_clv.groupby('Account No')['Amount'].transform('sum')

# Calculate total number of transactions per customer
df_clv['total_transactions_per_customer'] = df_clv.groupby('Account No').transform('size')

# Calculate average revenue per transaction
df_clv['avg_revenue_per_transaction'] = df_clv['total_revenue_per_customer'] / df_clv['total_transactions_per_customer']

# Calculate average transaction frequency
df_clv['total_days'] = (df_clv['Date'].max() - df_clv['Date'].min()).days + 1  # Total number of days in the dataset
df_clv['avg_transaction_frequency'] = df_clv['total_transactions_per_customer'] / df_clv['total_days']

# Calculate CLV
df_clv['clv'] = df_clv['avg_revenue_per_transaction'] * df_clv['avg_transaction_frequency']

print(df_clv.head())



        Date Timestamp   Account No  Balance  Amount  Third Party Account No  \
0 2023-01-01     00:00  678330503.0   2971.0  1584.0                     NaN   
1 2023-01-01     00:00  472213568.0   3792.0  1950.0                     NaN   
2 2023-01-01     00:00  472213568.0   3012.0  -780.0             283027736.0   
3 2023-01-01     00:00  283027736.0   1787.0   780.0             472213568.0   
4 2023-01-01     00:00  624500124.0   3226.0  1825.0                     NaN   

     Third Party Name  total_revenue_per_customer  \
0  Westport Care Home                     1267.32   
1    Barbiee Boutique                    -2281.85   
2                 NaN                    -2281.85   
3                 NaN                    -1075.81   
4            Fat Face                     2175.27   

   total_transactions_per_customer  avg_revenue_per_transaction  total_days  \
0                              423                     2.996028         340   
1                              213        

In [69]:
#print(df_clv[df_clv['Account No'] == 678330503].groupby('Account No').size())

#print(df_clv[df_clv['Account No'] == 678330503])

test_df = df_cleaned[df_cleaned['Amount'] > 0]
print(test_df['Third Party Name'].unique())


['Westport Care Home' 'Barbiee Boutique' nan 'Fat Face' 'Lavender Primary'
 'A Cut Above' 'Tesco' 'Brilliant Brushes' 'Costa Coffee'
 'Premier Finance' 'North Face' 'Craftastic' 'Fitted Stitch' 'Selfridges'
 'Starbucks' 'The Crown' 'Cass Art' 'Green Park Academy' 'Rose & Crown'
 'Kings Arms' 'Sunny Care Nursery' 'A Yarn Story' 'Pets Corner'
 'Lavender Fields' 'Victoria Park' 'Sainsbury' 'Hobby Lobby' 'Hobbycraft'
 'Lloyds Pharmacy' 'Boots' 'University College Hospital' 'LBG' 'Head'
 'CPA' 'Matalan' 'Happy Days Home' 'CeX' 'Pets at Home' 'Stitch By Stitch'
 'Jollyes' 'Collector Cave' 'Vision Express' 'Green Park'
 'Remedy plus care' 'Halifax' 'Gamestation' 'Grand Union BJJ']
